In [1]:
!pip install gdown
!gdown --id 1-1EjRHAq-4nhXIJ9-OobWmMXavl1B4zn

/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1-1EjRHAq-4nhXIJ9-OobWmMXavl1B4zn
From (redirected): https://drive.google.com/uc?id=1-1EjRHAq-4nhXIJ9-OobWmMXavl1B4zn&confirm=t&uuid=1f1d2578-5a44-4093-98b4-dc831c76992a
To: /kaggle/working/data.pkl
100%|███████████████████████████████████████| 1.24G/1.24G [00:09<00:00, 138MB/s]


In [2]:
import pickle
from tqdm import tqdm
import tensorflow as tf
import numpy as np

from preprocess import data_preparation
from preprocess import signal_cutter
from preprocess import data_filtering
from preprocess import resampling
from preprocess import RP_3D
from preprocess import resize_volume
from preprocess import RP_final_data
from preprocess import label_creator
from preprocess import including_data
from models import model_arch
from models import CompModel

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Read dictionary pkl file
with open('/kaggle/working/data.pkl', 'rb') as fp:
    dataset = pickle.load(fp)

In [4]:
ds = data_preparation(dataset)

# shortening
ds = signal_cutter(ds, d_length=1000)

# denoising
ds = data_filtering(ds)

# resampling
ds = resampling(ds, desired_points = 100)

100%|██████████| 10344/10344 [00:01<00:00, 7688.03it/s]


In [5]:
arr = {'NSR':'426783006', 'AF':'164889003', 'IAVB':'270492004', 'LBBB':'164909002', 'RBBB':'59118001', 'SB':'426177001', 'STach':'427084000'}
labels = label_creator(data_preparation(dataset), arr)
x_train, y_train, x_test, y_test = including_data(labels, ds)
# transformation and resizing
x_train = RP_final_data(x_train)

100%|██████████| 10344/10344 [00:00<00:00, 867596.15it/s]
10344it [00:08, 1239.09it/s]
10344it [00:04, 2152.89it/s]
100%|██████████| 5000/5000 [24:32<00:00,  3.39it/s]


In [6]:
# five-folds preparation

idx_lst = list(range(len(x_train)))

v1 = idx_lst[:1000]
t1 = idx_lst[1000:]

v2 = idx_lst[1000:2000]
t2 = idx_lst[:1000] + idx_lst[2000:]

v3 = idx_lst[2000:3000]
t3 = idx_lst[:2000] + idx_lst[3000:]

v4 = idx_lst[3000:4000]
t4 = idx_lst[:3000] + idx_lst[4000:]

v5 = idx_lst[4000:]
t5 = idx_lst[:4000]

In [7]:
model = model_arch()
model, lr_sch = CompModel(model)

history = model.fit(x_train[t3], y_train.values[t3],
            epochs=100,
            validation_data = (x_train[v3], y_train.values[v3]),
            callbacks = [lr_sch],
            verbose = 1)

Epoch 1/100


2023-11-06 18:17:23.165259: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: MutableGraphView::SortTopologically error: detected edge(s) creating cycle(s) {'Func/gradient_tape/model_3/conv_lstm2d_1/while/model_3/conv_lstm2d_1/while_grad/body/_367/input/_1025' -> 'gradient_tape/model_3/conv_lstm2d_1/while/model_3/conv_lstm2d_1/while_grad/body/_367/gradient_tape/model_3/conv_lstm2d_1/while/gradients/AddN', 'Func/gradient_tape/model_3/conv_lstm2d/while/model_3/conv_lstm2d/while_grad/body/_562/input/_1141' -> 'gradient_tape/model_3/conv_lstm2d/while/model_3/conv_lstm2d/while_grad/body/_562/gradient_tape/model_3/conv_lstm2d/while/gradients/AddN', 'model_3/conv_lstm2d_1/while/body/_179/model_3/conv_lstm2d_1/while/clip_by_value' -> 'model_3/conv_lstm2d_1/while/body/_179/model_3/conv_lstm2d_1/while/mul_3', 'model_3/conv_lstm2d_1/while/body/_179/model_3/conv_lstm2d_1/while/mul_2' -> 'model_3/conv_lstm2d_1/while/body/_179/model_3/conv_lstm2d_1/while/a

125/125 [==============================] - 209s 1s/step - loss: 0.4532 - acc: 0.3565 - val_loss: 0.7251 - val_acc: 0.1710 - lr: 0.0010
Epoch 2/100
125/125 [==============================] - 182s 1s/step - loss: 0.3031 - acc: 0.5890 - val_loss: 0.4858 - val_acc: 0.3260 - lr: 9.7724e-04
Epoch 3/100
125/125 [==============================] - 182s 1s/step - loss: 0.2605 - acc: 0.6480 - val_loss: 0.2448 - val_acc: 0.6750 - lr: 9.5499e-04
Epoch 4/100
125/125 [==============================] - 182s 1s/step - loss: 0.2374 - acc: 0.6645 - val_loss: 0.2119 - val_acc: 0.6890 - lr: 9.3325e-04
Epoch 5/100
125/125 [==============================] - 182s 1s/step - loss: 0.2177 - acc: 0.6820 - val_loss: 0.1932 - val_acc: 0.7100 - lr: 9.1201e-04
Epoch 6/100
125/125 [==============================] - 182s 1s/step - loss: 0.2041 - acc: 0.7092 - val_loss: 0.1974 - val_acc: 0.6970 - lr: 8.9125e-04
Epoch 7/100
125/125 [==============================] - 182s 1s/step - loss: 0.1936 - acc: 0.7147 - val_loss: 0

In [8]:
model.save('/kaggle/working/RP_ECGNet_Fold3.h5')